In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
from scipy import stats
# import warnings
# warnings.filterwarnings(action='ignore')

## Load Data

In [23]:
# Missing Values TEST
for col in df1.columns:
    pct_missing = np.mean(df1[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

Date - 0%
Time - 0%
Temp - 0%
Hum - 5%
TO - 16%


In [2]:
data_url = "./sensor_data_temp.log"
df = pd.read_csv(data_url, sep=" ", header=None, index_col=None)
df.head(1)

,0,1,2,3,4
0,1022-09-14,19:33:07,T=22.0,H=20.0,TO=45


In [3]:
# rename_columns
df.columns = ['Date', 'Time', 'Temp', 'Hum', 'TO']
df.head(1)

,Date,Time,Temp,Hum,TO
0,1022-09-14,19:33:07,T=22.0,H=20.0,TO=45


In [4]:
# Converting colums Date + Time to new column with pd.timestamp 'Datetime'
df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], errors="coerce")

In [16]:
# Function for checking if the datetime Timestamp is correct
def is_valid_date(datecheck):
    """ is_valid_date(datecheck)
    Checks if the date is valid. Returns a boolean value. Datasets with years before 2000 are not valid. Dates in the future are not valid.

    :param:     datecheck as pandas datetime with "%Y-%m-%d %H:%M:%S"
    :returns:   a boolean value
    :rtype:     bool
    """
    try:
        datecheck = pd.to_datetime(datecheck) 
        year = int(datecheck.strftime("%Y"))
        min_year = 2000
        now = datetime.today()
        return (min_year <= year and datecheck <= now)
    except:
        return print("Function is_valid_date: Something went wrong!")

In [36]:
def calculate_mean_timegap(df):
    """ calculate_mean_timegap(df)
    Calculates the mean timegap between times
 
    :param:     df
    :returns:   mean timegap as timestamp
    :rtype:     pd.timestamp
    """
    try: 
        first_index = 0
        last_index = len(df['Datetime'])-1
        end_time = df['Datetime'][last_index]

        # looking for first valid timestamp
        for index in df.index:
            start_time = df['Datetime'][first_index]
            if np.isnat(np.datetime64(str(start_time))):
                first_index+=1
            else:
                first_index = first_index
        print(f"First valid Timestamp: {start_time}, index: {first_index}")

        # looking for last valid timestamp
        for index in df.index:
            end_time = df['Datetime'][last_index]
            if np.isnat(np.datetime64(str(end_time))):
                last_index-=1
            else:
                last_index = last_index
        print(f"Last valid Timestamp: {end_time}, index: {last_index}")

        # calculating mean timegap between timestamps
        mean_timegap = (end_time - start_time) / (last_index - first_index)
        mean_timegap = pd.to_timedelta(str(mean_timegap)).round('1s')
        print(mean_timegap)
        return True
    except:
        return False

In [37]:
calculate_mean_timegap(df)

First valid Timestamp: 2022-09-14 19:48:14, index: 3
Last valid Timestamp: 2022-09-16 00:03:52, index: 334
0 days 00:05:07


True

In [30]:
# Function for replacing NaT with Timestamp values
def replace_nat(dataframe, mean_timegap):
    """ replace_nat(df, mean_timegap)
    Checks the dataframe for NaT. Replaces NaT with calculated Timestamp values.

    :param:     df = dataframe, mean_timegap = pandas datetime with "%Y-%m-%d %H:%M:%S"
    :returns:   a boolean value
    :rtype:     bool
    """    
    try:
        first_index = 0
        last_index = len(df['Datetime'])-1
        end_time = dataframe['Datetime'][last_index]
        nat_index = []

        # looking for first valid timestamp
        for index in dataframe.index:
            start_time = dataframe['Datetime'][first_index]
            if np.isnat(np.datetime64(str(start_time))):
                first_index+=1
            else:
                first_index = first_index
        print(f"First valid Timestamp: {start_time}, index: {first_index}")

        # looking for last valid timestamp
        for index in dataframe.index:
            end_time = dataframe['Datetime'][last_index]
            if np.isnat(np.datetime64(str(end_time))):
                last_index-=1
            else:
                last_index = last_index
        print(f"Last valid Timestamp: {end_time}, index: {last_index}")

        # checking for NaT
        print("Checking for NaT:")
        for index in dataframe.index:
            if np.isnat(np.datetime64(str(dataframe['Datetime'][index]))):
                print("df[" + str(index) + "]['Datetime']=" + str(dataframe['Datetime'][index]) + ": " + str(np.isnat(np.datetime64(str(dataframe['Datetime'][index])))))
                nat_index.append(index)
        print(f"Indices with NaT: {nat_index}")
            
        # replacing NaT with calculated Timestamps
        print("Replacing NaT with calculated Timestamps:")
        #for nat in reversed(nat_index):
        for nat in nat_index[::-1]:
            if (first_index < nat):
                if not np.isnat(np.datetime64(str(dataframe['Datetime'][nat - 1]))):
                    dataframe['Datetime'][nat] = dataframe['Datetime'][nat - 1] + mean_timegap
                else: 
                    dataframe['Datetime'][nat] = dataframe['Datetime'][nat + 1] - mean_timegap
            elif (first_index >= nat):
                dataframe['Datetime'][nat] = dataframe['Datetime'][first_index] - mean_timegap
                if first_index > 1:
                    first_index-=1
            else: 
                dataframe['Datetime'][nat] = dataframe['Datetime'][first_index] - mean_timegap
        
            print("Calculated Timestamp for: df[" + str(nat) + "]['Datetime']=" + str(dataframe['Datetime'][nat]))
        return dataframe                  
    except:
        return print("Function replace_nat: Something went wrong!")

In [31]:
#calculate_mean_timegap(df1)
df = replace_nat(df, calculate_mean_timegap(df))


NameError: name 'calculate_mean_timegap' is not defined

In [10]:
df.to_csv('./sensor_data_temp_corrected.log', index=False)

In [39]:
str1 = '2019-02-28 27:02:58'
print ('Your string: {}'.format(str1))
print ('Your string_to_convert type: {}'.format(type(str1)))

date1 = pd.to_datetime(str1, errors="coerce")
print ('Your new date is: {}'.format(date1))
print ('Your new type is: {}'.format(type(date1)))



Your string: 2019-02-28 27:02:58
Your string_to_convert type: <class 'str'>
Your new date is: NaT
Your new type is: <class 'pandas._libs.tslibs.nattype.NaTType'>


In [11]:
# Converting Date to datetime, invalid parsing will be set as NaT
# df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], errors="coerce")

print("Checking for valid date:")   
for index in df.index:    
    # Checking for valid dates
    if not is_valid_date(np.datetime64(df['Datetime'][index])):
        print("df[" + str(index) + "]['Datetime']=" + str(df['Datetime'][index]) + " is not valid! "+ str(df['Datetime'][index]))
        # Changing invalid Date to NaT
        df['Datetime'][index] = str("NaT")
        print("df[" + str(index) + "]['Datetime']=" + str(df['Datetime'][index]))
        df = replace_nat(df, calculate_mean_timegap(df))
        

Checking for valid date:
df[162]['Datetime']=2032-09-15 09:31:05 is not valid! 2032-09-15 09:31:05
df[162]['Datetime']=NaT
First valid Timestamp: 2022-09-14 19:32:53, index: 0
Last valid Timestamp: 2022-09-16 00:08:59, index: 335
0 days 00:05:07.361194029
First valid Timestamp: 2022-09-14 19:32:53, index: 0
Last valid Timestamp: 2022-09-16 00:08:59, index: 335
Checking for NaT:
df[162]['Datetime']=NaT: True
Indices with NaT: [162]
Replacing NaT with calculated Timestamps:
Calculated Timestamp for: df[162]['Datetime']=2022-09-15 09:31:11


In [27]:
class CheckDataQuality:
    def __init__(self, dataframe):
        self.dataframe = dataframe
    
    valid_temp = [-20, 50]  # Wertebereich Temperatur
    valid_hum = [0, 100]    # Wertebereich Luftfeuchtigkeit
      
    def rename_columns(self):
        """ rename_columns(df)
        Renames the columns of the Dataset to ['Date', 'Time', 'Temp', 'Hum', 'TO']
    
        :param:     df
        :returns:   a boolean value
        :rtype:     bool
        """
        try:
            self.dataframe.columns = ['Date', 'Time', 'Temp', 'Hum', 'TO']
            return True
        except:
            return False

    def create_datetime(self):
        """ create_datetime(df)
        Creates a new column 'Datetime' from 'Date' and 'Time'. Drops columns 'Date' and 'Time'.
    
        :param:     df
        :returns:   a boolean value
        :rtype:     bool
        """
        try:
            # Converting colums Date + Time to new column with pd.timestamp 'Datetime'
            self.dataframe['Datetime'] = pd.to_datetime(self.dataframe['Date'] + ' ' + self.dataframe['Time'], errors="coerce")
            # Dropping Columns Date and Time
            self.dataframe = self.dataframe.drop(columns=['Date', 'Time'])
            return True
        except:
            return False

    def replace_nat(self):
        """ replace_nat(df, mean_timegap)
        Checks the dataframe for NaT. Replaces NaT with calculated Timestamp values.

        :param:     df = dataframe, mean_timegap = pandas datetime with "%Y-%m-%d %H:%M:%S"
        :returns:   a boolean value
        :rtype:     bool
        """    
        try:
            first_index = 0
            last_index = len(self.dataframe['Datetime'])-1
            end_time = self.dataframe['Datetime'][last_index]
            nat_index = []

            # looking for first valid timestamp
            for index in self.dataframe.index:
                start_time = self.dataframe['Datetime'][first_index]
                if np.isnat(np.datetime64(str(start_time))):
                    first_index+=1
                else:
                    first_index = first_index
            print(f"First valid Timestamp: {start_time}, index: {first_index}")

            # looking for last valid timestamp
            for index in self.dataframe.index:
                end_time = self.dataframe['Datetime'][last_index]
                if np.isnat(np.datetime64(str(end_time))):
                    last_index-=1
                else:
                    last_index = last_index
            print(f"Last valid Timestamp: {end_time}, index: {last_index}")

            # calculating mean timegap between timestamps
            mean_timegap = (end_time - start_time) / (last_index - first_index)
            mean_timegap = pd.to_timedelta(str(mean_timegap)).round('1s')
            print(f"Mean Timegap: {mean_timegap}")

            # checking for NaT
            print("Checking for NaT...")
            for index in self.dataframe.index:
                if np.isnat(np.datetime64(str(self.dataframe['Datetime'][index]))):
                    print("df[" + str(index) + "]['Datetime']=" + str(self.dataframe['Datetime'][index]) + ": " + str(np.isnat(np.datetime64(str(self.dataframe['Datetime'][index])))))
                    nat_index.append(index)
            print(f"Indices with NaT: {nat_index}")
                
            # replacing NaT with calculated Timestamps
            print("Replacing NaT with calculated Timestamps...")
            #for nat in reversed(nat_index):
            for nat in nat_index[::-1]:
                if (first_index < nat):
                    if not np.isnat(np.datetime64(str(self.dataframe['Datetime'][nat - 1]))):
                        self.dataframe['Datetime'][nat] = self.dataframe['Datetime'][nat - 1] + mean_timegap
                    else: 
                        self.dataframe['Datetime'][nat] = self.dataframe['Datetime'][nat + 1] - mean_timegap
                elif (first_index >= nat):
                    self.dataframe['Datetime'][nat] = self.dataframe['Datetime'][first_index] - mean_timegap
                    if first_index > 1:
                        first_index-=1
                else: 
                    self.dataframe['Datetime'][nat] = self.dataframe['Datetime'][first_index] - mean_timegap
            
                print("Calculated Timestamp for: df[" + str(nat) + "]['Datetime']=" + str(self.dataframe['Datetime'][nat]))
            return True                  
        except:
            return False
        
    # def is_valid_date(datecheck):
    #     """ is_valid_date(datecheck)
    #     Checks if the date is valid. Returns a boolean value. Datasets with years before 2000 are not valid. Dates in the future are not valid.

    #     :param:     datecheck as pandas datetime with "%Y-%m-%d %H:%M:%S"
    #     :returns:   a boolean value
    #     :rtype:     bool
    #     """
    #     try:
    #         datecheck = pd.to_datetime(datecheck) 
    #         year = int(datecheck.strftime("%Y"))
    #         min_year = 2000
    #         now = datetime.today()
    #         return (min_year <= year and datecheck <= now)
    #     except:
    #         return False

    def check_valid_date(self):
        """ check_valid_date(self)
        Checks if the date is valid (calls is_valid_date). 

        :param:     df
        :returns:   a boolean value
        :rtype:     bool
        """ 
        min_year = 2000
        now = datetime.today()
        try:
            # 4. Checking for valid dates
            for index in self.dataframe.index:    
                # Checking for valid dates
                datecheck = (self.dataframe['Datetime'][index])
                year = int(datecheck.strftime("%Y"))
                if not (min_year <= year and datecheck <= now):
                    print("df[" + str(index) + "]['Datetime']=" + str(self.dataframe['Datetime'][index]) + " is not valid! "+ str(self.dataframe['Datetime'][index]))
                    # Changing invalid Date to NaT
                    self.dataframe['Datetime'][index] = str("NaT")
                    print("df[" + str(index) + "]['Datetime']=" + str(self.dataframe['Datetime'][index]))
                    # Calling replace_nat function
                    print("Calling replace_nat Function")         
                    self.dataframe = replace_nat(self.dataframe)
                    print("Returning from replace_nat function")
            print("All Dates are valid!")
            return True
        except:
            print("Dates are not valid!")
            return False

    def change_index(self):
        try:
            print("Setting Datetime as index...")
            self.dataframe.set_index(pd.DatetimeIndex(df['Datetime'], inplace=True))
            print("Reindexing columns...")
            column_names = ["Datetime", "Temp", "Hum", "TO"]
            self.dataframe = self.dataframe.reindex(columns=column_names)
            return True
        except:
            return False
               
    # Format Data Columns
    def format_data_columns(self):
        try:
            # Replacing Strings in Temp and Hum, Dropping TO
            print("Formatting Data Columns...")
            print("String Extraction...")
            self.dataframe['Temp'] = self.dataframe['Temp'].str.replace('T=', '')
            self.dataframe['Hum'] = self.dataframe['Hum'].str.replace('H=', '')
            print("Dropping Column TO...")
            self.dataframe = self.dataframe.drop(columns=['TO'])
            # Convert each value of the column to a string
            print("Converting from string to float...")
            self.dataframe['Temp'] = pd.to_numeric(self.dataframe['Temp'], errors='coerce')
            self.dataframe['Hum'] = pd.to_numeric(self.dataframe['Hum'], errors='coerce')
            # Replace empty string ('') with np.nan before convertion
            self.dataframe['Temp'] = self.dataframe['Temp'].replace('',np.nan)
            self.dataframe['Hum'] = self.dataframe['Hum'].replace('',np.nan)
            print("Done!")
            return True
        except:
            return False

    def replace_outliers(self): # Check: https://neuraldatascience.io/5-eda/data_cleaning.html
        try:
            print("Checking for outliers...")
            # Checking Z-Score < 3
            #if np.abs(stats.zscore(self.dataframe['Hum']) < 3):
            #outliers = np.abs(stats.zscore(self.dataframe['Hum']) < 3)
            # Replacing Outliers with Z-Score > 3
            #self.dataframe = self.dataframe.mask(self.dataframe.sub(self.dataframe.mean()).div(self.dataframe.std()).abs().gt(3))
            self.dataframe = self.dataframe.mask(((self.dataframe < self.dataframe.quantile(0.05)) or (self.dataframe > self.dataframe.quantile(0.95))), np.nan, axis=1)
            print("Done!")
            return True
        except:
            return False

    def check_duplicates(self):
        if duplicates:
            print("Duplicates found!")
        else:
            print("No duplicates found!")
            
    def check_timestamps(self):
        if timestamps:
            print("Incorrect timestamps found!")
        else:
            print("No incorrect timestamps found!")
            
    def check_missingvalues(self):
        if missingvalues:
            print("Missing values found!")
        else:
            print("No missing values found!")

    def check_outliers(self):
        if outliers:
            print("Outliers found!")
        else:
            print("No outliers found!")
            
    def check_sensorquality(self):
        if sensorquality:
            print("Drifting values found!")
        else:
            print("No drifting values found!")
            
    def check_wrongvalues(self):
        if wrongwalues:
            print("Wrong values found!")
        else:
            print("No wrong values found!")

    def visualize(self):
        if visualize:
            print("Visualizing Data!")
        else:
            print("Something went wrong!")
    
    def export_new_file(self):
        try:
            print("Exporting corrected file...")
            self.dataframe.to_csv('./sensor_data_temp_corrected.log', index=False)
            return True
        except:
            return False

    def start(self):
        rename_columns(self)

        

In [4]:
test1 = CheckDataQuality(df)

In [5]:
def check_date():
    print("TEST")
    try:
        print("Starting check_date function...")
        # 1. Rename columns
        if test1.rename_columns():
            print("Columns renamed!")
        else:
            print("Function rename_columns: Something went wrong!")

        # 2. Create Datetime column, drop Date and Time
        if test1.create_datetime():
            print("Datetime created. Date and Time dropped!")
        else:
            print("Function create_datetime: Something went wrong!") 

        # 3. Replace NaT with Timestamp
        if test1.replace_nat():
            print("All NaT values replaced!")
        else:
            print("Function replace_nat: Something went wrong!")
        
        # 4. Checking for valid dates
        if test1.check_valid_date():
            print("All Dates are valid!")
        else:
            print("Function check_valid_date: Something went wrong!")

        # 5. Export new File
        if test1.export_new_file():
            print("File exported!")
        else:
            print("Function export_new_file: Something went wrong!")
    except:
        print("Function check_date: Something went wrong!")

In [28]:
test3 = CheckDataQuality(df)

In [30]:
test3.rename_columns()
test3.create_datetime()
test3.replace_nat()
test3.check_valid_date()



First valid Timestamp: 2022-09-14 19:32:53, index: 0
Last valid Timestamp: 2022-09-16 00:08:59, index: 335
Mean Timegap: 0 days 00:05:07
Checking for NaT...
df[162]['Datetime']=NaT: True
Indices with NaT: [162]
Replacing NaT with calculated Timestamps...
Calculated Timestamp for: df[162]['Datetime']=2022-09-15 09:31:11
All Dates are valid!


C:\Users\jirid\AppData\Local\Temp\ipykernel_3396\2651597816.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.dataframe['Datetime'][nat] = self.dataframe['Datetime'][nat - 1] + mean_timegap


True

In [31]:
test3.change_index()
test3.format_data_columns()
test3.replace_outliers()
test3.export_new_file()

Setting Datetime as index...
Formatting Data Columns...
String Extraction...
Dropping Column TO...
Converting from string to float...
Done!
Checking for outliers...
Exporting corrected file...


C:\Users\jirid\AppData\Local\Temp\ipykernel_3396\2651597816.py:194: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version. Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  self.dataframe = self.dataframe.mask(((self.dataframe < self.dataframe.quantile(0.05)) or (self.dataframe > self.dataframe.quantile(0.95))), np.nan, axis=1)


True

In [26]:
test3.dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336 entries, 0 to 335
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Temp      320 non-null    float64       
 1   Hum       320 non-null    float64       
 2   Datetime  336 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(2)
memory usage: 8.0 KB
